# Sparse Gaussian Process: an exploration

**Computational Statistics Project (2021-22). Jaime Enríquez Ballesteros.**

"*Gaussian process (GP) models are flexible probabilistic nonparametric models for regression,
classification and other tasks. Unfortunately they suffer from computational intractability for
large data sets. Over the past decade there have been many different approximations
developed to reduce this cost. Most of these can be termed global approximations, in that they
try to summarize all the training data via a small set of support points. A different approach is
that of local regression, where many local experts account for their own part of space. In this
project we are interested to study the regimes in which these different approaches work well
or fail, and then apply a new sparse GP approximation which is a combination of both the
global and local approaches, and look extremely promising.*"

General imports:

In [ ]:
import GPflow.gpflow as gpflow
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from GPflow.gpflow.config import default_float, default_jitter
from GPflow.gpflow.utilities import print_summary, set_trainable, to_default_float
from GPflow.gpflow.models import maximum_log_likelihood_objective, training_loss_closure
from GPflow.gpflow.ci_utils import ci_niter

from sklearn.metrics import mean_squared_error

from timeit import default_timer as timer

import scipy

## Introduction

This project covers different techniques which might be used when wanting to use a Gaussian Process Regression while having a big dataset. As it will be shown, this is a big problem since as the dataset being used gets bigger, the problem becomes computationally intractable.

The solution at hand is that of Sparse Gaussian Process Regression, which try to approximate the posterior distribution of the real Gaussian Process Regression model without loosing too much information about the data.Two main groups can be distinguished: local and global methods. Several models have been proposed which combine "*the best of both worlds*": a combination between both. All of these will be explained in the following sections.

The project is focused in understanding GPR and SGPR, with the help of the articles by *Snelson and Ghahramani(2007)* and *Titsias (2009)* (found at the bottom of the document). The principal focus is made on the former. A reproducibility objective seems interesting with regard to this articles, as well as an expansion with an external database.

The programming framework used to model Gaussian Processes is *GPFlow* (https://www.gpflow.org/): similar to GPyTorch, only implemented through TensorFlow.

As a sidenote about the framework, we have to take into mind that not EVERY possibility is available among the huge world of Gaussian Processes. It is also a young framework, with little documentation.

To compress these ideas in a single list of objectives, we have:

**Objectives of the project:**
1. Understand Sparse Gaussian Process Regression on big volume datasets.
2. Understand how they are implemented in GPFlow library and the different models we can find.
3. Try to reproduce experimentation on *Snelson & Ghahramani* (2007) and *Titsias* (2009) to understand how these models work.
4. Use an external database(s) to confirm the characteristics of each method.
5. Implement different schemes to place inducing points.

### General Functions implemented

In [ ]:

def plot_univariate(model, color, ax, lims=[-1,11], show_xs=True, show_ind_locations=True):
    x = model.data[0]
    y = model.data[1]
    xx = np.linspace(lims[0], lims[1], 100).reshape(-1, 1)
    mu, var = model.predict_y(xx)
    ax.plot(xx, mu, color, lw=2)
    ax.fill_between(
        xx[:, 0],
        mu[:, 0] - 1.96 * np.sqrt(var[:, 0]),
        mu[:, 0] + 1.96 * np.sqrt(var[:, 0]),
        color=color,
        alpha=0.2,
    )
    if show_xs:
        ax.plot(x, y, "k,", mew=2)
    if show_ind_locations:
        try:
            ax.plot(np.array(model.inducing_variable.variables[0]),
                    np.repeat(min(mu[:, 0] - 1.96 * np.sqrt(var[:, 0])) - 1, N_ind),
                    'rx')
        except:
            pass
    # Plot title
    ax.set_title(type(model).__name__)
    ax.set_xlim(lims[0], lims[1])


def create_models(data,
                  inducing_variable,
                  optimize=True,
                  opt=gpflow.optimizers.Scipy(),  # BFGS
                  niter=100,
                  verbose=False
                  ):
    ndim=data[0].shape[1]
    kernel = gpflow.kernels.SquaredExponential()
    kernel = gpflow.kernels.Matern32(lengthscales=np.ones(n_dim))
    m1 = gpflow.models.GPR(data, kernel=kernel)

    kernel = gpflow.kernels.Matern32(lengthscales=np.ones(n_dim))
    m2 = gpflow.models.SGPR(
        data, kernel=kernel, inducing_variable=inducing_variable
    )
    set_trainable(m2.inducing_variable, False)

    kernel = gpflow.kernels.Matern32(lengthscales=np.ones(n_dim))
    m3 = gpflow.models.GPRFITC(
        data, kernel=kernel, inducing_variable=inducing_variable
    )
    set_trainable(m3.inducing_variable, False)

    kernel = gpflow.kernels.Matern32(lengthscales=np.ones(n_dim))
    m4 = gpflow.models.GPRPITC(
        data, kernel=kernel, inducing_variable=inducing_variable
    )
    set_trainable(m4.inducing_variable, False)

    kernel = gpflow.kernels.Matern32(lengthscales=np.ones(n_dim))
    m5 = gpflow.models.GPRPIC(
        data, kernel=kernel, inducing_variable=inducing_variable
    )
    set_trainable(m5.inducing_variable, False)


    models = [m2,m3]

    if verbose:
        for model in models:
            print_summary(model)

    times = []
    if optimize:
        # Optimize models
        for model in models:
            print("Starting training of ", type(model).__name__)
            loss_closure = training_loss_closure(model, data)
            start = timer()
            opt.minimize(
                loss_closure,
                variables=model.trainable_variables,
                options=dict(maxiter=ci_niter(100))
            )
            end = timer()
            print("Finished training of ", type(model).__name__)
            times.append(end-start)


    return models, times


# m.predict_f -> returns the mean and variance of 𝑓 at the points Xnew
# m.predict_f_samples -> returns samples of the latent function.
# m.predict_y -> returns the mean and variance of a new data point (that is, it includes the noise variance).
def mse_test(model, test_data):
    test_set, test_labels = test_data
    X_t = tf.convert_to_tensor(test_set, dtype=default_float())
    mean, var = model.predict_y(X_t)
    return mean_squared_error(mean, test_labels)



# m.predict_density -> returns the log density of the observations Ynew at Xnew.
def nlpd_test(model, test_data):
    N = test_data[0].shape[0]
    return 1/N*np.sum(model.predict_log_density(test_data))


def error_vs_time(times, mses, nlpds,
                  markers=['.', '.', 'o', '+'],
                  colors=['black', 'red', 'blue', 'black'],
                  mfcs=[False, False, True, False],
                  model_names=['GPR', 'SGPR', 'FITC', 'PITC']):

    fig, axs = plt.subplots(1, 2, figsize=(16,8))

    # MSE vs time/s
    for i in range(len(times)):
        axs[0].scatter(times[i],
                       mses[i],
                       marker=markers[i],
                       facecolors='none' if mfcs[i] else colors[i],
                       edgecolors=colors[i],
                       label=model_names[i])

    axs[0].legend()
    axs[0].set_ylabel("MSE")
    axs[0].set_xlabel("time/s")
    axs[0].set_title("MSE vs time")

    # NLPD vs time/s
    for i in range(len(times)):
        axs[1].scatter(times[i],
                       nlpds[i],
                       marker=markers[i],
                       facecolors='none' if mfcs[i] else colors[i],
                       edgecolors=colors[i],
                       label=model_names[i])

    axs[1].legend()
    axs[1].set_ylabel("NLPD")
    axs[1].set_xlabel("time/s")
    axs[1].set_title("NLPD vs time")


## Introduction to Sparse Gaussian Process Regression

### Brief recap on Gaussian Process

$\TODO$

### Gaussian Process Regression

$\TODO$

### Why do we need Sparse GPR?

$\TODO$

## Types of Sparse Gaussian Process Regression models.

From the multitude of papers and models one can find about Sparse GPR models, this project focuses on the proposals presented in *Snelson & Ghahramani (2007)*. Here, a distinction is made between:
* **Global approaches**: where the idea behind is to use a set of support points which can summarize the whole dataset. A similar procedure to the original GPR is used using only the support points. This is the reason for the term *global*.

* **Local approaches**: where data is grouped such that a GPR model is deployed for each of this blocks. The posterior distribution for a test point will correspond to the block to which this point belongs to. Such is, a *local* approach.

In the paper mentioned above, a recap of two global methods are presented. These are the FIC (Fully Independent Conditional) and FITC (Fully Independent (Training) Conditional) approximations. Given a set of inducing points $\overline{f}$, the GP prior is expressed as:

$$
p(f,f_t) = \int d\overline{f} \text{  } p(f,f_T | \overline{f})  \text{  } p(\overline{f})
$$

From here, it is assumed that $f$ and $f_T$ are independent given $\overline{f}$:

$$
p(f,f_T) \approx q(f,f_T) = \int d\overline{f} \text{  } q(f_T | \overline{f}) \text{  } q(f | \overline{f})  \text{  } p(\overline{f})
$$

The FITC and FIC differ on the different following assumptions made on the training and test conditionals.

### FIC
For the first of the Fully Independent global approximations, the assumption is made that both the training and test conditionals are fully independent s.t. $q(f|\overline{f})=\prod\limits_n{p(f_n|\overline{f})}$ and $q(f_T|\overline{f})=\prod\limits_t{p(f_t|\overline{f})}$. 

As a result the prior and predictive distributions found are the ones found in the first row of Table 1.

### FITC
This second global approximation only considers the *training* data points to be independent. We find the same conditional for training points as before: $q(f|\overline{f})=\prod\limits_n{p(f_n|\overline{f})}$; but, now, the test conditional: $q(f_T|\overline{f})=p(f_T|\overline{f})$.

This will transform into a different covariance prior in the testing points slots, as shown in Table 1.

![Table 1](img/Table_1.png)

### Global + Local approach

Both global and local approaches present very good qualities. They also show disadvantages which are, somehow, complementary of each other. This is the main motivation to search for a method which combines both approaches.

As a first proposal, in *Snelson & Ghahramani (2007)* we find the PITC (Partially Independent Training Conditional).

### PITC

First proposed in *Quiñonera and Rasmussen (2005)*, the PITC method follows the steps of FITC. As a first step, the training points are divided into blocks $  X_{B_s}, f_{B_s} $ being $s \in S$ each of the blocks.

PITC assumes *partial independence* for the training points. Independence is only considered for points which do not belong to the same block. Following the same notation as FI(T)C, the training and test conditionals are: $q(f|\overline{f})=\prod\limits_s^S{p(f_{B_s}|\overline{f})}$ and $q(f_T|\overline{f})=p(f_T|\overline{f})$.

These conditionals probabilities make us understand that the covariance prior will have to be composed by a *block-diagonal* taken of the original training points. The blocks of the block-diagonal matrix being covariance sub-matrices of the original covariance matrix (minus the new sparse diagonal $Q_N$). The corresponding formulas for the predictive distribution (on test point $f_*$) are shown in Table 2.

### PIC

A new approach is developed in *Snelson & Ghahramani (2007)* following the steps of PITC. The motivation for it is that PITC is still to similar to FITC (as we can see by comparing the predictive distributions). One can not expect too much difference between the two. Another important issue is that the blocking scheme in the training points marginalizes the testing points into a block of their own. Meaning that the test points are related to the training points only through the inducing points. An issue we wanted to avoid when combining global and local methods.

The new approach, named PIC, treats training and testing points and considers them jointly for the blocking scheme. This results in a joint conditional: $q(f, f_T|\overline{f})=p(f_{B_s}, f_T|\overline{f}) \prod\limits_s^{S-1}{p(f_{B_s}|\overline{f})}$.

The corresponding predictive distribution is shown in Table 2.

![](img/Table_2.png)

Of course, two questions come to mind about the methods shown above.
* **Where to put / How many inducing points** to obtain the best model possible?
* **How big / How many blocks** should be appointed?

Usually, the location of inducing points is decided randomly. Although an interesting proposal is choosing the corresopnding location of inducing points based on the clusters formed in the local-part of the method.

Regarding how many inducing points to appoint, the only requirement is that this number should be much smaller than the original number of data points ($M<<N$).

The size of the blocks is usually the same for every block. So the size and number of blocks usually have a direct relation. According to *Quiñonera and Rasmussen (2005)*, "*A reasonable choice, recommended by Tresp (1999) may be to choose k = n/m blocks.*" i.e. the number of total points divided by the number of inducing points.

It is important to mention that when the number of inducing points or blocks in the model is increased greatly, it will tend to a GP Regresion model (since $Q=K$). 
On the other spectrum, when block sizes tend to one we obtain the FIC model. Furthermore, when the inducing points tend to zero, we are left with the original local GP predictor.

## Implementation of Local, PITC and PIC methods in GPFlow context

## Simple Guassian Regression Example (d=1)

In [ ]:
np.random.seed(42)
N = 1000
N_t = 100
N_ind = 10
X = np.random.rand(N, 1) * 10
Y = np.sin(X) + 0.9 * np.cos(X * 1.6) + np.random.randn(*X.shape) * 0.4
Xtest = np.random.rand(N_t, 1) * 10
_ = plt.plot(X, Y, "kx", mew=2)
plt.show()

In [ ]:
data = (
    tf.convert_to_tensor(X, dtype=default_float()),
    tf.convert_to_tensor(Y, dtype=default_float()),
)
X_ind = X[0:N-1:int(N/N_ind)]
inducing_variable = tf.convert_to_tensor(X_ind, dtype=default_float())

In [ ]:
# Create models
models, _ = create_models(data, inducing_variable)

GPR, SGPR, FITC = models
LocalGPR = gpflow.models.LocalGPR(data,gpflow.kernels.SquaredExponential(), num_blocks=10)
LocalGPR.optimize()


In [ ]:
# Plot results
f, ax = plt.subplots(3, 2, figsize=(12, 9), sharex=False, sharey=False)
plot_univariate(GPR, "C0", ax[0, 0], show_xs=True)
plot_univariate(SGPR, "C1", ax[0, 1], show_xs=True)
plot_univariate(FITC, "C2", ax[1, 0], show_xs=True)
#plot_univariate(PITC, "C3", ax[1, 1], show_xs=True, lims=[-1,6])
# plot_univariate(PIC, "C4", ax[2, 0], show_xs=False)
plot_univariate(LocalGPR, "C5", ax[2, 1], show_xs=True)
plt.show()

## Comparison between Guassian Process Models

## Reproduction of previous experiments found in *Snelson & Ghahramani* (2007) and *Titsias* (2009)

### Kin40k dataset

40,000 records describing the location of a robotic arm as a function of an 8-dimensional control input

In [ ]:
# Load data
kin40k = np.loadtxt("data/kin40k/kin40k_train_data.asc") # Toy example of kin40k
kin40k_l = np.loadtxt("data/kin40k/kin40k_train_labels.asc").reshape(-1, 1) # Toy example of kin40k
kin40k_test = np.loadtxt("data/kin40k/kin40k_test_data.asc", skiprows=29000) # Toy example of kin40k
kin40k_test_l = np.loadtxt("data/kin40k/kin40k_test_labels.asc", skiprows=29000).reshape(-1, 1) # Toy example of kin40k
N = len(kin40k)
N_t = len(kin40k_test)
N_ind = len(kin40k) * 0.01  
n_dim = kin40k.shape[1]

kin40k_data = (
    tf.convert_to_tensor(kin40k, dtype=default_float()),
    tf.convert_to_tensor(kin40k_l, dtype=default_float()),
)
kin40k_test_data = (
    tf.convert_to_tensor(kin40k_test, dtype=default_float()),
    tf.convert_to_tensor(kin40k_test_l, dtype=default_float()),
)
kin40k_ind = kin40k[0:N-1:int(N/N_ind)]
kin40k_ind = tf.convert_to_tensor(kin40k_ind, dtype=default_float())

# https://github.com/GPflow/GPflow/issues/1606
models, times = create_models(kin40k_data, kin40k_ind)
# GPR, SGPR, FITC, PITC, PIC = models
SGPR, FITC = models

LocalGPR = gpflow.models.LocalGPR(kin40k_data,  gpflow.kernels.SquaredExponential(lengthscales=np.ones(n_dim)), num_blocks=100)
times.append(sum(LocalGPR.optimize()))

# Evaluation of kin40k
kin40k_mses = [#mse_test(GPR, kin40k_test_data),
               mse_test(SGPR, kin40k_test_data),
               mse_test(FITC, kin40k_test_data),
               # mse_test(PITC, kin40k_test_data),
               mse_test(LocalGPR, kin40k_test_data)]

kin40k_nlpd = [#nlpd_test(GPR, kin40k_test_data),
               nlpd_test(SGPR, kin40k_test_data),
               nlpd_test(FITC, kin40k_test_data),
               #nlpd_test(PITC, kin40k_test_data)
               nlpd_test(LocalGPR, kin40k_test_data)]


error_vs_time(times, kin40k_mses, kin40k_nlpd)
plt.show()

### Abalone dataset

Predicting the age of abalone from physical measurements. The age of abalone is determined by
cutting the shell through the cone, staining it, and counting the number of rings through a
microscope -- a boring and time-consuming task. Other measurements, which are easier to obtain,
are used to predict the age.

In [ ]:

abalone = pd.read_csv('data/abalone/abalone.data', header=None)
N = len(abalone)
test_pct = .2
abalone_test = abalone.iloc[:int(N*test_pct),:]
abalone_test_l = abalone_test.iloc[:,-1]
abalone_test = abalone_test.iloc[:,:-1].drop(0, axis=1)

abalone = abalone.iloc[int(N*test_pct):,:]
abalone_l = abalone.iloc[:,-1]
abalone = abalone.iloc[:,:-1].drop(0, axis=1)

N = len(abalone)
N_t = len(abalone_test)
N_ind = len(abalone) * 0.01
n_dim = abalone.shape[1]

abalone_data = (
    tf.convert_to_tensor(abalone, dtype=default_float()),
    tf.convert_to_tensor(abalone_l, dtype=default_float()),
)
abalone_test_data = (
    tf.convert_to_tensor(abalone_test, dtype=default_float()),
    tf.convert_to_tensor(abalone_test_l, dtype=default_float()),
)
abalone_ind = abalone[0:N-1:int(N/N_ind)]
abalone_ind = tf.convert_to_tensor(abalone_ind, dtype=default_float())

# Training of SARCOS
models, times = create_models(abalone_data, gpflow.kernels.Matern32(lengthscales=np.ones(n_dim)), abalone_ind)
#GPR, SGPR, FITC, PITC, PIC = models
SGPR, FITC = models

LocalGPR = gpflow.models.LocalGPR(abalone_data,  gpflow.kernels.SquaredExponential(lengthscales=np.ones(n_dim)), num_blocks=100)
times.append(sum(LocalGPR.optimize()))

# Evaluation of SARCOS
abalone_mses = [#mse_test(GPR, abalone_test_data),
               mse_test(SGPR, abalone_test_data),
               mse_test(FITC, abalone_test_data),
               #mse_test(PITC, abalone_test_data),
               mse_test(LocalGPR, abalone_test_data)]

abalone_nlpd = [#nlpd_test(GPR, abalone_test_data),
               nlpd_test(SGPR, abalone_test_data),
               nlpd_test(FITC, abalone_test_data),
               #nlpd_test(PITC, abalone_test_data),
               nlpd_test(LocalGPR, abalone_test_data)]

error_vs_time(times, abalone_mses, abalone_nlpd)
plt.show()

### SARCOS dataset

The data relates to an inverse dynamics problem for a seven degrees-of-freedom SARCOS anthropomorphic robot arm.
The task is to map from a 21-dimensional input space (7 joint positions, 7 joint velocities, 7 joint accelerations)
to the corresponding 7 joint torques.
There are 44,484 training examples and 4,449 test examples. The first 21 columns are the input variables,
and the 22nd column is used as the target variable.

In [ ]:
sarcos = scipy.io.loadmat('data/sarcos/sarcos_inv.mat')['sarcos_inv']
sarcos_l = sarcos[:,-1]
sarcos = sarcos[:,:-1]
sarcos_test = scipy.io.loadmat('data/sarcos/sarcos_inv_test.mat')['sarcos_inv_test']
sarcos_test_l = sarcos_test[:,-1]
sarcos_test = sarcos_test[:,:-1]

N = len(sarcos)
N_t = len(sarcos_test)
N_ind = len(sarcos) * 0.01 
n_dim = sarcos.shape[1]

sarcos_data = (
    tf.convert_to_tensor(sarcos, dtype=default_float()),
    tf.convert_to_tensor(sarcos_l, dtype=default_float()),
)
sarcos_test_data = (
    tf.convert_to_tensor(sarcos_test, dtype=default_float()),
    tf.convert_to_tensor(sarcos_test_l, dtype=default_float()),
)
sarcos_ind = sarcos[0:N-1:int(N/N_ind)]
sarcos_ind = tf.convert_to_tensor(sarcos_ind, dtype=default_float())

# Training of SARCOS
models, times = create_models(sarcos_data, gpflow.kernels.Matern32(lengthscales=np.ones(n_dim)), sarcos_ind)
#GPR, SGPR, FITC, PITC = models
SGPR, FITC= models

LocalGPR = gpflow.models.LocalGPR(abalone_data,  gpflow.kernels.SquaredExponential(lengthscales=np.ones(n_dim)), num_blocks=100)
times.append(sum(LocalGPR.optimize()))

# Evaluation of SARCOS
sarcos_mses = [#mse_test(GPR, sarcos_test_data),
               mse_test(SGPR, sarcos_test_data),
               mse_test(FITC, sarcos_test_data),
               #mse_test(PITC, sarcos_test_data),
               mse_test(LocalGPR, sarcos_test_data)]

sarcos_nlpd = [#nlpd_test(GPR, sarcos_test_data),
               nlpd_test(SGPR, sarcos_test_data),
               nlpd_test(FITC, sarcos_test_data),
               #nlpd_test(PITC, sarcos_test_data),
               nlpd_test(LocalGPR, sarcos_test_data)]

error_vs_time(times, sarcos_mses, sarcos_nlpd)
plt.show()

## External dataset

### References

1. Snelson and Ghahramani. (2007). Local and global sparse Gaussian process approximations. Artificial Intelligence and Statistics 11 (AISTATS).http://proceedings.mlr.press/v2/snelson07a/snelson07a.pdf
2. Titsias. (2009). Variational Learning of Inducing Variables in Sparse Gaussian Processes. Journal of Machine Learning Research - Proceedings Track. 5. 567-574. http://proceedings.mlr.press/v5/titsias09a/titsias09a.pdf
3. Quiñonera and Rasmussen (2005). A Unifying View of Sparse Approximate Gaussian Process Regression.https://www.jmlr.org/papers/v6/quinonero-candela05a.html